In [13]:
import os

# Create the emissions folder if it doesn't exist
if not os.path.exists('./emissions'):
	os.makedirs('./emissions')

#improt file in "Sad-Brain-Analytics-Hackathon\carbon_tracker.py"
%run carbon_tracker.py
import carbon_tracker
carbon_tracker.start_tracker()

In [14]:
sum = 0
for i in range(10000000):
    sum += i
print(sum)

carbon_tracker.update_project_name("Test 555")


49999995000000


In [15]:
sum = 0
for i in range(10000000):
    sum += i
    
    for i in range(10):
        sum += i


print(sum)

carbon_tracker.update_project_name("Test 002")

50000445000000
